In [ ]:
import os
import sys

original_sys_path = sys.path.copy()

try:
    import module
except ImportError:
    print('Module system not found. Make sure the required python packages are available in the current environment.')
else:
    sys.path = original_sys_path
    await module.purge(force=True)
    print('Loading required modules from EESSI...')
    await module.load('AITW-stiffness/1.0.0-foss-2023a')

    import os
    import sys

    for extra in os.getenv('PYTHONPATH').split(':'):
        if extra not in sys.path:
            sys.path.append(extra)

In [1]:
import json
from pathlib import Path

import ipywidgets as wdg
from aitw_stiffness_surrogate.cli import aitw_stiffness

from common import STIFFNESS_DEFAULTS

import utilities

In [ ]:
widgets = utilities.get_widgets_from_click_function(
    aitw_stiffness,
    override_defaults=STIFFNESS_DEFAULTS
)

title = wdg.HTML(value="<h2>AITW - Surrogate Stiffness evaluation</h2>")
display(title)
widget_box = wdg.VBox(list(widgets.values()))
display(widget_box)

generate_button = wdg.Button(description="Evaluate Stiffness", button_style="success")
output_area = wdg.Output()

@utilities.widget_button_running(running_style="warning", running_text="Evaluating...")
def on_generate_button_clicked(b):
    with output_area:
        output_area.clear_output()
        args = utilities.get_click_args_from_widgets(aitw_stiffness, widgets)
        print("Running AITW Stiffness evaluation...")
        for arg in args:
            print(f'- {arg}')
        
        output = Path(args[1])
        output.parent.mkdir(parents=True, exist_ok=True)

        aitw_stiffness(args, standalone_mode=False)
        # Display content of the output file
        with output.open() as f:
            data = json.load(f)
        display(data)

generate_button.on_click(on_generate_button_clicked)
display(generate_button)
display(output_area)